In [274]:
import pandas as pd
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import random
import os 
import numpy as np 
import tensorflow as tf
# Set a random seed for reproducibility
seed_value = 42
os.environ['PYTHONHASHSEED'] = str(seed_value)
random.seed(seed_value)
np.random.seed(seed_value)
tf.random.set_seed(seed_value)
import pandas as pd 
from transformers import AutoTokenizer, TFAutoModel
import pandas as pd 
from sklearn.model_selection import train_test_split
import tensorflow
from tensorflow.keras.layers import Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dropout, Dense, Bidirectional
from tensorflow.keras.layers import Layer
import tensorflow.keras.backend as K

In [275]:
## Load data 

synthetic_user_data = pd.read_csv("/Users/nyzy/Library/CloudStorage/GoogleDrive-nitinmali999@gmail.com/.shortcut-targets-by-id/1yEbTjc1DwwTDd2CB86Il6F-3DhFUn8pU/Data Mining/data/processed_datasets/synthetic_data_half_cleaned_mac_full_v1.csv")
import ast 
df = synthetic_user_data.copy()
# Fill missing values if necessary
df.fillna({'Course Taken': 'no course','course title': 'no course'}, inplace=True)
df["Course Taken"]=df["Course Taken"].apply(lambda x:x.replace("NAN","No Course"))

df["Course Skills"]=df["Course Skills"].apply(lambda x:ast.literal_eval(x))
df["Course Skills"].apply(lambda x: x if len(x)!=0 else "no skills")
#df["Course title"]=df["Course title"].apply(lambda x:x.replace("NAN","No Course"))
def set_to_string(skill_set):
    if not skill_set:  # Checks if the set is empty
        return 'no skills' 
    # Check if skill_set is a string that needs to be evaluated
    if isinstance(skill_set, str):
        try:
            # Try to evaluate the string as a set
            skill_set = ast.literal_eval(skill_set)
        except (ValueError, SyntaxError):
            # Handle cases where the string is not a valid set
            pass  # You might want to return a default value or handle this case as needed
    # Convert to string if it's a set or list
    if isinstance(skill_set, (set, list)):
        return ', '.join(skill_set)
    return skill_set 

set_columns = ['Current Skills', 'Next Skills', 'Skill Gap', 'Course Skills']
for col in set_columns:
    print(col)
    #df[col]=df[col].apply(lambda x:ast.literal_eval(x))
    df[col] = df[col].apply(set_to_string)





#df = pd.read_csv("/Users/nyzy/Library/CloudStorage/GoogleDrive-nitinmali999@gmail.com/.shortcut-targets-by-id/1yEbTjc1DwwTDd2CB86Il6F-3DhFUn8pU/Data Mining/data/processed_datasets/df_embedding_bert.csv")

# Loading the JSON file back into a dictionary
import json
Job_progression_dictionary_file_path = "/Users/nyzy/Library/CloudStorage/GoogleDrive-nitinmali999@gmail.com/.shortcut-targets-by-id/1yEbTjc1DwwTDd2CB86Il6F-3DhFUn8pU/Data Mining/data/processed_datasets/job_progression_dictionary.json"

with open(Job_progression_dictionary_file_path, 'r') as json_file:
    sorted_full_job_progression_dict_lower = json.load(json_file)

#print(sorted_full_job_progression_dict_lower)
# Output: {'name': 'John', 'age': 30, 'city': 'New York'}




Current Skills
Next Skills
Skill Gap
Course Skills


In [276]:
df = df[df["Next Job"]!="unmatched"]
df = df.drop(columns=['Unnamed: 0.1', 'Unnamed: 0'])
df = df[df["random next job"]!=1].drop_duplicates()

In [277]:
df["Course Taken"].unique()

array(['python', 'undefined', 'data science', 'database management', 'c',
       'data quality management', 'data analysis', 'java algorithms',
       'sql', 'data mining', 'continuous integration',
       'artificial intelligence', 'data visualization',
       'machine learning', 'database design', 'apache airflow',
       'microsoft sql', 'databricks', 'information systems',
       'data architecture', 'business analytics', 'b testing',
       'user experience design', 'docker', 'big data', 'deep learning',
       'pyspark', 'deployment', 'database administration',
       'cloud computing', 'amazon aws', 'opencv',
       'natural language processing', 'apache spark', 'credit analysis',
       'r programming language', 'pytorch', 'java', 'automation',
       'azure synapse analytics', 'microsoft azure', 'bitcoin',
       'text mining', 'aws databases', 'matlab', 'predictive modeling',
       'tensorflow', 'linear algebra'], dtype=object)

In [236]:
#job prediction

# load embeddings 

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Add 'start' and 'end' tokens to each target sequence
df['Next Job'] = df['Next Job'].apply(lambda x: 'start ' + x + ' end')


loaded_embeddings = np.load('/Users/nyzy/Library/CloudStorage/GoogleDrive-nitinmali999@gmail.com/.shortcut-targets-by-id/1yEbTjc1DwwTDd2CB86Il6F-3DhFUn8pU/Data Mining/data/processed_datasets/all_embeddings_v1.npz')
#'Next Skills', 'Skill Gap', 'course title', 'Course Skills'
feature_columns = ['Current Job', 'Current Skills' ]
# Stack embeddings horizontally (axis=1)
all_embeddings = np.hstack((loaded_embeddings[each] for each in feature_columns)) # Add other embeddings as needed

tokenizer = Tokenizer()
tokenizer.fit_on_texts(df['Next Job'])  # Replace with your target column
target_seqs = tokenizer.texts_to_sequences(df['Next Job'])
target_seqs_padded = pad_sequences(target_seqs, padding='post')

# Vocabulary size for the output
target_vocab_size = len(tokenizer.word_index) + 1

# Shift target sequences for the decoder's training
decoder_input_data = target_seqs_padded[:, :-1]  # all except last token
decoder_target_data = target_seqs_padded[:, 1:]  


#y_course = loaded_embeddings['Course Taken_embedding']

from sklearn.model_selection import train_test_split

# Assuming 'target' is your target array
X_train, X_test, y_job_train, y_job_test = train_test_split(all_embeddings, target_seqs_padded, test_size=0.2, random_state=42)
decoder_input_train, decoder_input_test = train_test_split(decoder_input_data, test_size=0.2, random_state=42)
decoder_target_train, decoder_target_test = train_test_split(decoder_target_data, test_size=0.2, random_state=42)

# Reshape input embeddings for the model
# This depends on the expected input shape of your model. 
# For example, if your model expects 3D input (samples, timesteps, features):
X_train_reshaped = X_train.reshape(X_train.shape[0], len(feature_columns), -1)
X_test_reshaped = X_test.reshape(X_test.shape[0], len(feature_columns), -1)

import tensorflow as tf

X_train_tensor = tf.convert_to_tensor(X_train_reshaped, dtype=tf.float32)
X_test_tensor = tf.convert_to_tensor(X_test_reshaped, dtype=tf.float32)
decoder_input_train_tensor = tf.convert_to_tensor(decoder_input_train, dtype=tf.float32)
decoder_input_test_tensor = tf.convert_to_tensor(decoder_input_test, dtype=tf.float32)
decoder_target_train_tensor = tf.convert_to_tensor(decoder_target_train, dtype=tf.float32)
decoder_target_test_tensor = tf.convert_to_tensor(decoder_target_test, dtype=tf.float32)


# Verify the new shapes
print("X_train_tensor shape:", X_train_tensor.shape)
print("X_test_tensor shape:", X_test_tensor.shape)

print("decoder_input_train_tensor shape:", decoder_input_train_tensor.shape)
print("decoder_input_test_tensor shape:", decoder_input_test_tensor.shape)
print("decoder_target_train_tensor shape:", decoder_target_train_tensor.shape)
print("decoder_target_test_tensor shape:", decoder_target_test_tensor.shape)


/var/folders/17/gws30jys63j9xx5l0mwr5r4r0000gq/T/ipykernel_58179/3891330826.py:16: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  all_embeddings = np.hstack((loaded_embeddings[each] for each in feature_columns)) # Add other embeddings as needed


X_train_tensor shape: (4319, 3, 768)
X_test_tensor shape: (1080, 3, 768)
decoder_input_train_tensor shape: (4319, 6)
decoder_input_test_tensor shape: (1080, 6)
decoder_target_train_tensor shape: (4319, 6)
decoder_target_test_tensor shape: (1080, 6)


In [ ]:
#course prediction

# load embeddings 

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Add 'start' and 'end' tokens to each target sequence
df['Course Taken'] = df['Course Taken'].apply(lambda x: 'start ' + x + ' end')


loaded_embeddings = np.load('/Users/nyzy/Library/CloudStorage/GoogleDrive-nitinmali999@gmail.com/.shortcut-targets-by-id/1yEbTjc1DwwTDd2CB86Il6F-3DhFUn8pU/Data Mining/data/processed_datasets/all_embeddings_v1.npz')
#'Next Skills', 'Skill Gap', 'course title', 'Course Skills'
feature_columns = ['Current Job', 'Current Skills', 'Next Job' ]
# Stack embeddings horizontally (axis=1)
all_embeddings = np.hstack((loaded_embeddings[each] for each in feature_columns)) # Add other embeddings as needed

tokenizer = Tokenizer()
tokenizer.fit_on_texts(df['Course Taken'])  # Replace with your target column
target_seqs = tokenizer.texts_to_sequences(df['Course Taken'])
target_seqs_padded = pad_sequences(target_seqs, padding='post')

# Vocabulary size for the output
target_vocab_size = len(tokenizer.word_index) + 1

# Shift target sequences for the decoder's training
decoder_input_data = target_seqs_padded[:, :-1]  # all except last token
decoder_target_data = target_seqs_padded[:, 1:]  


#y_course = loaded_embeddings['Course Taken_embedding']

from sklearn.model_selection import train_test_split

# Assuming 'target' is your target array
X_train, X_test, y_job_train, y_job_test = train_test_split(all_embeddings, target_seqs_padded, test_size=0.2, random_state=42)
decoder_input_train, decoder_input_test = train_test_split(decoder_input_data, test_size=0.2, random_state=42)
decoder_target_train, decoder_target_test = train_test_split(decoder_target_data, test_size=0.2, random_state=42)

# Reshape input embeddings for the model
# This depends on the expected input shape of your model. 
# For example, if your model expects 3D input (samples, timesteps, features):
X_train_reshaped = X_train.reshape(X_train.shape[0], len(feature_columns), -1)
X_test_reshaped = X_test.reshape(X_test.shape[0], len(feature_columns), -1)

import tensorflow as tf

X_train_tensor = tf.convert_to_tensor(X_train_reshaped, dtype=tf.float32)
X_test_tensor = tf.convert_to_tensor(X_test_reshaped, dtype=tf.float32)
decoder_input_train_tensor = tf.convert_to_tensor(decoder_input_train, dtype=tf.float32)
decoder_input_test_tensor = tf.convert_to_tensor(decoder_input_test, dtype=tf.float32)
decoder_target_train_tensor = tf.convert_to_tensor(decoder_target_train, dtype=tf.float32)
decoder_target_test_tensor = tf.convert_to_tensor(decoder_target_test, dtype=tf.float32)


# Verify the new shapes
print("X_train_tensor shape:", X_train_tensor.shape)
print("X_test_tensor shape:", X_test_tensor.shape)

print("decoder_input_train_tensor shape:", decoder_input_train_tensor.shape)
print("decoder_input_test_tensor shape:", decoder_input_test_tensor.shape)
print("decoder_target_train_tensor shape:", decoder_target_train_tensor.shape)
print("decoder_target_test_tensor shape:", decoder_target_test_tensor.shape)


In [263]:
df['Next Skills']

0        start algorithmic trading, computer science, p...
2        start ad hoc analysis, business intelligence, ...
3        start quantitative analysis, artificial intell...
4        start computer science, data dictionary, busin...
5        start computer science, statistical, business ...
                               ...                        
39859      start business development, problem solving end
39916    start market analysis, business requirements, ...
39931    start computer science, technology solutions, ...
39940          start http, data driven decision making end
39944    start machine learning, analytics, artificial ...
Name: Next Skills, Length: 5399, dtype: object

In [271]:
df['Next Skills']

0        start algorithmic trading, computer science, p...
2        start ad hoc analysis, business intelligence, ...
3        start quantitative analysis, artificial intell...
4        start computer science, data dictionary, busin...
5        start computer science, statistical, business ...
                               ...                        
39859      start business development, problem solving end
39916    start market analysis, business requirements, ...
39931    start computer science, technology solutions, ...
39940          start http, data driven decision making end
39944    start machine learning, analytics, artificial ...
Name: Next Skills, Length: 5399, dtype: object

In [278]:
# skills prediction 

# load embeddings 

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Add 'start' and 'end' tokens to each target sequence
df['Next Skills'] = df['Next Skills'].apply(lambda x: 'start,' + x + ',end')


loaded_embeddings = np.load('/Users/nyzy/Library/CloudStorage/GoogleDrive-nitinmali999@gmail.com/.shortcut-targets-by-id/1yEbTjc1DwwTDd2CB86Il6F-3DhFUn8pU/Data Mining/data/processed_datasets/all_embeddings_v1.npz')
#'Next Skills', 'Skill Gap', 'course title', 'Course Skills'
feature_columns = ['Current Job', 'Current Skills', 'Next Job' ]
# Stack embeddings horizontally (axis=1)
all_embeddings = np.hstack((loaded_embeddings[each] for each in feature_columns)) # Add other embeddings as needed

tokenizer = Tokenizer(split=',')
tokenizer.fit_on_texts(df['Next Skills'])  # Replace with your target column
target_seqs = tokenizer.texts_to_sequences(df['Next Skills'])
target_seqs_padded = pad_sequences(target_seqs, padding='post')

# Vocabulary size for the output
target_vocab_size = len(tokenizer.word_index) + 1

# Shift target sequences for the decoder's training
decoder_input_data = target_seqs_padded[:, :-1]  # all except last token
decoder_target_data = target_seqs_padded[:, 1:]  


#y_course = loaded_embeddings['Course Taken_embedding']

from sklearn.model_selection import train_test_split

# Assuming 'target' is your target array
X_train, X_test, y_job_train, y_job_test = train_test_split(all_embeddings, target_seqs_padded, test_size=0.2, random_state=42)
decoder_input_train, decoder_input_test = train_test_split(decoder_input_data, test_size=0.2, random_state=42)
decoder_target_train, decoder_target_test = train_test_split(decoder_target_data, test_size=0.2, random_state=42)

# Reshape input embeddings for the model
# This depends on the expected input shape of your model. 
# For example, if your model expects 3D input (samples, timesteps, features):
X_train_reshaped = X_train.reshape(X_train.shape[0], len(feature_columns), -1)
X_test_reshaped = X_test.reshape(X_test.shape[0], len(feature_columns), -1)

import tensorflow as tf

X_train_tensor = tf.convert_to_tensor(X_train_reshaped, dtype=tf.float32)
X_test_tensor = tf.convert_to_tensor(X_test_reshaped, dtype=tf.float32)
decoder_input_train_tensor = tf.convert_to_tensor(decoder_input_train, dtype=tf.float32)
decoder_input_test_tensor = tf.convert_to_tensor(decoder_input_test, dtype=tf.float32)
decoder_target_train_tensor = tf.convert_to_tensor(decoder_target_train, dtype=tf.float32)
decoder_target_test_tensor = tf.convert_to_tensor(decoder_target_test, dtype=tf.float32)


# Verify the new shapes
print("X_train_tensor shape:", X_train_tensor.shape)
print("X_test_tensor shape:", X_test_tensor.shape)

print("decoder_input_train_tensor shape:", decoder_input_train_tensor.shape)
print("decoder_input_test_tensor shape:", decoder_input_test_tensor.shape)
print("decoder_target_train_tensor shape:", decoder_target_train_tensor.shape)
print("decoder_target_test_tensor shape:", decoder_target_test_tensor.shape)


/var/folders/17/gws30jys63j9xx5l0mwr5r4r0000gq/T/ipykernel_58179/2200069783.py:16: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  all_embeddings = np.hstack((loaded_embeddings[each] for each in feature_columns)) # Add other embeddings as needed


X_train_tensor shape: (4319, 3, 768)
X_test_tensor shape: (1080, 3, 768)
decoder_input_train_tensor shape: (4319, 33)
decoder_input_test_tensor shape: (1080, 33)
decoder_target_train_tensor shape: (4319, 33)
decoder_target_test_tensor shape: (1080, 33)


In [279]:

from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Dense, Embedding

# Model parameters
bert_embedding_dim = 768
latent_dim = 256  # Dimensionality of the LSTM layer
target_vocab_size = len(tokenizer.word_index) + 1  # Vocabulary size for the output

# Encoder
encoder_inputs = Input(shape=(len(feature_columns), bert_embedding_dim))  # 6 features, each with a BERT embedding of size 768
encoder_lstm = LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder_lstm(encoder_inputs)
encoder_states = [state_h, state_c]

# Decoder
decoder_inputs = Input(shape=(None,))  # 'None' allows the model to handle variable length sequences
decoder_embedding = Embedding(target_vocab_size, latent_dim)
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_dense = Dense(target_vocab_size, activation='softmax')

# Embed and decode the sequence
dec_emb = decoder_embedding(decoder_inputs)
decoder_outputs, _, _ = decoder_lstm(dec_emb, initial_state=encoder_states)
decoder_outputs = decoder_dense(decoder_outputs)

# Define the model
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

# Compile the model
model.compile(optimizer='rmsprop', loss='sparse_categorical_crossentropy')



In [280]:
# Train the model
history = model.fit(
    [X_train_tensor, decoder_input_train_tensor], 
    np.expand_dims(decoder_target_train_tensor, -1),  # Add an extra dimension to the target
    batch_size=32,
    epochs=20,
    validation_data=(
        [X_test_tensor, decoder_input_test_tensor], 
        np.expand_dims(decoder_target_test_tensor, -1)
    )
)


Epoch 1/20
135/135 [==============================] - 23s 140ms/step - loss: 1.2690 - val_loss: 0.9860
Epoch 2/20
135/135 [==============================] - 18s 131ms/step - loss: 0.9235 - val_loss: 0.8659
Epoch 3/20
135/135 [==============================] - 20s 151ms/step - loss: 0.8081 - val_loss: 0.7457
Epoch 4/20
135/135 [==============================] - 22s 164ms/step - loss: 0.6870 - val_loss: 0.6386
Epoch 5/20
135/135 [==============================] - 21s 156ms/step - loss: 0.5773 - val_loss: 0.5338
Epoch 6/20
135/135 [==============================] - 20s 148ms/step - loss: 0.4850 - val_loss: 0.4482
Epoch 7/20
135/135 [==============================] - 20s 147ms/step - loss: 0.4057 - val_loss: 0.3782
Epoch 8/20
135/135 [==============================] - 19s 138ms/step - loss: 0.3415 - val_loss: 0.3181
Epoch 9/20
135/135 [==============================] - 18s 134ms/step - loss: 0.2883 - val_loss: 0.2858
Epoch 10/20
135/135 [==============================] - 19s 139ms/step - l

In [281]:
model.summary()


Model: "model_46"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_62 (InputLayer)          [(None, None)]       0           []                               
                                                                                                  
 input_61 (InputLayer)          [(None, 3, 768)]     0           []                               
                                                                                                  
 embedding_18 (Embedding)       (None, None, 256)    81920       ['input_62[0][0]']               
                                                                                                  
 lstm_28 (LSTM)                 [(None, 256),        1049600     ['input_61[0][0]']               
                                 (None, 256),                                              

In [282]:
# Encoder model for inference
encoder_model = Model(encoder_inputs, encoder_states)


In [283]:
# Decoder setup for inference
decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

dec_emb2 = decoder_embedding(decoder_inputs)  # Reuse the same embedding layer
decoder_outputs2, state_h2, state_c2 = decoder_lstm(dec_emb2, initial_state=decoder_states_inputs)
decoder_states2 = [state_h2, state_c2]
decoder_outputs2 = decoder_dense(decoder_outputs2)

decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs2] + decoder_states2
)


In [284]:
def decode_sequence(input_seq):
    # Encode the input sequence to get the internal states
    states_value = encoder_model.predict(input_seq)

    # Generate empty target sequence of length 1 with only the start token
    target_seq = np.zeros((1, 1))
    start_token_index = tokenizer.word_index['start']  # Assuming you have a 'start' token
    target_seq[0, 0] = start_token_index

    # Sampling loop to generate sequence
    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)

        # Sample a token and add its corresponding word to the decoded sequence
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_word = tokenizer.index_word[sampled_token_index]
        if sampled_word != 'end':  # Assuming you have an 'end' token
            decoded_sentence += ' ' + sampled_word

        # Exit condition: either hit max length or find stop token
        if sampled_word == 'end' or len(decoded_sentence) > 50:
            stop_condition = True

        # Update the target sequence (of length 1) and states
        target_seq = np.zeros((1, 1))
        target_seq[0, 0] = sampled_token_index
        states_value = [h, c]

    return decoded_sentence


In [285]:
tokenizer.word_index

{'start': 1,
 'end': 2,
 ' analytics': 3,
 ' sql': 4,
 ' data science': 5,
 ' python': 6,
 'computer science': 7,
 ' project management': 8,
 ' machine learning': 9,
 ' problem solving': 10,
 ' computer science': 11,
 ' data management': 12,
 ' data analysis': 13,
 ' data quality': 14,
 ' data governance': 15,
 ' business intelligence': 16,
 ' big data': 17,
 ' c': 18,
 ' data modeling': 19,
 ' algorithms': 20,
 ' data integration': 21,
 ' data engineering': 22,
 ' data architecture': 23,
 ' business requirements': 24,
 ' data visualization': 25,
 ' data warehousing': 26,
 ' java': 27,
 ' visualization': 28,
 ' data pipelines': 29,
 ' data processing': 30,
 ' data driven decision making': 31,
 ' statistical': 32,
 ' technology solutions': 33,
 ' artificial intelligence': 34,
 ' data mining': 35,
 ' data structures': 36,
 ' power bi': 37,
 'project management': 38,
 'analytics': 39,
 ' tensorflow': 40,
 'machine learning': 41,
 ' scala': 42,
 ' pytorch': 43,
 ' data strategy': 44,
 ' da

In [286]:
def decode_sequence(input_seq):
    # Encode the input sequence to get the internal states
    states_value = encoder_model.predict(input_seq)

    # Generate empty target sequence of length 1 with only the start token
    target_seq = np.zeros((1, 1))
    start_token_index = tokenizer.word_index['start']  # Assuming you have a 'start' token
    target_seq[0, 0] = start_token_index

    # Sampling loop to generate sequence
    stop_condition = False
    decoded_sequence = []
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_word = tokenizer.index_word.get(sampled_token_index, '?')

        # Check for 'end' token or maximum length
        if sampled_word == 'end' or len(decoded_sequence) > 50:  # 'end' is your end-of-sequence token
            stop_condition = True
        else:
            decoded_sequence.append(sampled_word)

        # Update the target sequence (of length 1) and states
        target_seq = np.zeros((1, 1))
        target_seq[0, 0] = sampled_token_index
        states_value = [h, c]

    # Join the words with a comma
    decoded_sentence = ', '.join(decoded_sequence)
    return decoded_sentence


In [287]:
for i in range(100):  # Generate predictions for the first 10 test samples
    input_seq = X_test_tensor[i: i + 1]
    decoded_sentence = decode_sequence(input_seq)
    print('Predicted sequence:', decoded_sentence)


1/1 [==============================] - 0s 28ms/step
Predicted sequence: business intelligence,  business reporting,  business requirements,  analytics,  project management,  c,  data warehousing,  visualization
1/1 [==============================] - 0s 29ms/step
Predicted sequence: computer science,  data dictionary,  business intelligence,  analytics,  data architecture,  data structures,  data quality,  data warehousing,  data integration
1/1 [==============================] - 0s 29ms/step
Predicted sequence: computer science,  problem solving
1/1 [==============================] - 0s 30ms/step
Predicted sequence: computer science,  data dictionary,  data mapping,  data modeling,  databricks,  data mining,  analytics,  big data,  cloud technologies,  data management,  sql,  data science,  azure data factory,  data warehousing
1/1 [==============================] - 0s 29ms/step
Predicted sequence: computer science,  statistical,  business intelligence,  analytics,  data analysis,  pow

In [218]:
from sklearn.metrics import accuracy_score

def strip_target_variable(target_value):
    target_value=target_value.replace("start","").replace("end","").lstrip().rstrip()
    return target_value
# Generate predictions for the test set
y_pred = [decode_sequence(X_test_tensor[i: i + 1]) for i in range(len(X_test_tensor))]

# You need to have your test target sequences in text format for comparison
y_true = [" ".join(tokenizer.sequences_to_texts([y])) for y in y_job_test]

y_pred = list(map(strip_target_variable,y_pred))
y_true = list(map(strip_target_variable,y_true))
# Compute accuracy
accuracy = accuracy_score(y_true, y_pred)
print("Accuracy:", accuracy)


1/1 [==============================] - 0s 32ms/step
Accuracy: 0.3277777777777778


In [219]:
print("Accuracy:", accuracy)

Accuracy: 0.3277777777777778


In [221]:
X_test

array([[-0.19868153, -0.20300885, -0.28546572, ..., -0.52706814,
        -0.04692927, -0.0289749 ],
       [-0.04485014, -0.01365658, -0.07303683, ..., -0.22763899,
        -0.16574879,  0.13514139],
       [-0.27258956, -0.04662251, -0.05476945, ..., -0.40453902,
        -0.2310756 , -0.00149782],
       ...,
       [ 0.04258528, -0.3384775 , -0.3686056 , ..., -0.29378515,
        -0.2608714 ,  0.03722385],
       [-0.1505295 , -0.05431197, -0.38077262, ..., -0.29865944,
        -0.35075116,  0.25280032],
       [-0.13431166, -0.41167888, -0.30107138, ..., -0.54847085,
        -0.10559174, -0.12287349]], dtype=float32)

In [223]:
for x,y in zip(y_true,y_pred):
    print(x,y)

business intelligence manager business intelligence manager
data architect data architect
chief information officer chief information officer
data architect data architect
business intelligence analyst data analyst
machine learning architect senior machine learning engineer
director of product management data architect
senior data engineer data architect
data manager data architect
principal data scientist director of data science
chief information officer data architect
chief product officer data architect
director of analytics data architect
director of data engineering director of product management
senior data scientist director of data science
principal data scientist director of data science
senior data engineer data architect
director of analytics director of analytics
director of data engineering chief information officer
data product manager chief information officer
data architect data architect
machine learning engineer data architect
senior data scientist principal data sci

In [103]:
def prepare_initial_input(user_current_job, other_features):
    # Combine the user's current job with other features
    combined_text = user_current_job + ' ' + ' '.join(other_features)
    
    # Generate BERT embeddings for the combined text
    user_input_embeddings = get_bert_embeddings([combined_text])  # Assuming a function like before
    user_input_embeddings = user_input_embeddings.reshape(1, 6, -1)  # Reshape to match the model's input shape
    
    return user_input_embeddings


'data architect'

In [189]:
tokenizer_b = AutoTokenizer.from_pretrained("bert-base-uncased")
model = TFAutoModel.from_pretrained("bert-base-uncased")

def get_embedding(text):
    inputs = tokenizer_b(text, return_tensors="tf", padding=True, truncation=True, max_length=512)
    outputs = model(inputs)
    return tf.reduce_mean(outputs.last_hidden_state, 1).numpy()[0]  # Mean pooling

def batch_encode(texts, batch_size=32):
    embeddings = []
    for i in range(0, len(texts), batch_size):
        batch = texts[i:i+batch_size]
        inputs = tokenizer_b(batch, return_tensors="tf", padding=True, truncation=True, max_length=512)
        outputs = model(inputs)
        batch_embeddings = tf.reduce_mean(outputs.last_hidden_state, 1).numpy()
        embeddings.extend(batch_embeddings)
    return embeddings
text_columns = ['Current Job', 'Next Job', 'Current Skills', 'Next Skills', 'Skill Gap', 'course title', 'Course Skills', 'Course Taken']
df_user = df[df["Current Job"]=="data analyst"][0:1]
df_user["Next Skills"]="unmatched"
df_user["Skill Gap"]="unmatched"
df_user["Course Taken"]="unmatched"
df_user["Course Skills"]="unmatched"
embeddings_dict_user = {}
for col in text_columns:
    embeddings_dict_user[col] = batch_encode(df_user[col].tolist())

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions w

In [200]:

feature_columns = ['Current Job', 'Current Skills', 'Next Skills', 'Skill Gap', 'course title', 'Course Skills']
# Stack embeddings horizontally (axis=1)
user_embedding = np.hstack((embeddings_dict_user[each] for each in feature_columns)) 

/var/folders/17/gws30jys63j9xx5l0mwr5r4r0000gq/T/ipykernel_58179/921930486.py:3: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  user_embedding = np.hstack((embeddings_dict_user[each] for each in feature_columns))


In [201]:
user_embedding.shape
user_embedding = user_embedding.reshape(1, 6, -1)

In [202]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences


X_user_tensor = tf.convert_to_tensor(user_embedding, dtype=tf.float32)
next_job = decode_sequence(X_test_tensor[0:1])
next_job

1/1 [==============================] - 0s 31ms/step


' start data architect'

In [184]:
X_test_tensor.shape

TensorShape([200, 6, 768])

In [183]:
X_user_tensor.shape

TensorShape([1, 6, 768])

In [174]:
df_user

,Unnamed: 0,Current Job,Next Job,Current Skills,Next Skills,Skill Gap,course title,Course Taken,Course Skills,random next job
34,34,data analyst,start head of big data end,"analytics, business analysis, data visualizati...",unmatched,unmatched,intro data data science,unmatched,unmatched,0


In [175]:
df_user = df[df["Current Job"]=="data scientist"][0:1]

In [176]:
df_user

,Unnamed: 0,Current Job,Next Job,Current Skills,Next Skills,Skill Gap,course title,Course Taken,Course Skills,random next job
1,1,data scientist,start senior data scientist end,"statistical, machine learning, business analyt...","natural language processing, pytorch, data ana...","pytorch, computer vision, python, docker, data...",no course,docker,no skills,0


In [149]:
df_user

,Unnamed: 0,Current Job,Next Job,Current Skills,Next Skills,Skill Gap,course title,Course Taken,Course Skills,random next job
34,34,data analyst,start head of big data end,"analytics, business analysis, data visualizati...","data strategy, data quality, business intellig...","data management, data privacy laws, data scien...",intro data data science,data science,"data structures, data science",0


In [ ]:
y_true